In [222]:
#initial imports for data pipeline 
import pandas as pd
import numpy as np
import seaborn as sns

In [223]:

price_index_df=pd.read_csv(r'C:\Users\wesle\Downloads\County_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv')
price_index_df.columns

Index(['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName',
       'State', 'Metro', 'StateCodeFIPS', 'MunicipalCodeFIPS', '2000-01-31',
       ...
       '2024-09-30', '2024-10-31', '2024-11-30', '2024-12-31', '2025-01-31',
       '2025-02-28', '2025-03-31', '2025-04-30', '2025-05-31', '2025-06-30'],
      dtype='object', length=315)

In [224]:
price_index_df.head(20)

,RegionID,SizeRank,RegionName,RegionType,StateName,State,Metro,StateCodeFIPS,MunicipalCodeFIPS,2000-01-31,...,2024-09-30,2024-10-31,2024-11-30,2024-12-31,2025-01-31,2025-02-28,2025-03-31,2025-04-30,2025-05-31,2025-06-30
0,3101,0,Los Angeles County,county,CA,CA,"Los Angeles-Long Beach-Anaheim, CA",6,37,217016.713301,...,9.121329e+05,9.174954e+05,9.218029e+05,9.253645e+05,9.245597e+05,9.206221e+05,9.134864e+05,9.079587e+05,9.027568e+05,8.984227e+05
1,139,1,Cook County,county,IL,IL,"Chicago-Naperville-Elgin, IL-IN-WI",17,31,154684.720407,...,3.213713e+05,3.217450e+05,3.221581e+05,3.228574e+05,3.237260e+05,3.246195e+05,3.251359e+05,3.254122e+05,3.253988e+05,3.254104e+05
2,1090,2,Harris County,county,TX,TX,"Houston-The Woodlands-Sugar Land, TX",48,201,113170.599399,...,2.921608e+05,2.917820e+05,2.913160e+05,2.910759e+05,2.908786e+05,2.905736e+05,2.897515e+05,2.887002e+05,2.876827e+05,2.866218e+05
3,2402,3,Maricopa County,county,AZ,AZ,"Phoenix-Mesa-Chandler, AZ",4,13,147729.658573,...,4.825138e+05,4.817623e+05,4.810772e+05,4.804818e+05,4.795186e+05,4.779998e+05,4.757882e+05,4.737405e+05,4.716651e+05,4.697450e+05
4,2841,4,San Diego County,county,CA,CA,"San Diego-Chula Vista-Carlsbad, CA",6,73,218116.075033,...,9.575502e+05,9.568249e+05,9.570212e+05,9.575403e+05,9.576494e+05,9.571071e+05,9.549489e+05,9.517400e+05,9.468407e+05,9.415196e+05
5,1286,5,Orange County,county,CA,CA,"Los Angeles-Long Beach-Anaheim, CA",6,59,256482.128905,...,1.167088e+06,1.173815e+06,1.180490e+06,1.187105e+06,1.191890e+06,1.195068e+06,1.194865e+06,1.192885e+06,1.188138e+06,1.182613e+06
6,581,6,Kings County,county,NY,NY,"New York-Newark-Jersey City, NY-NJ-PA",36,47,213808.102756,...,9.054754e+05,9.078185e+05,9.109907e+05,9.155746e+05,9.185237e+05,9.206937e+05,9.225964e+05,9.274965e+05,9.326564e+05,9.378175e+05
7,2964,7,Miami-Dade County,county,FL,FL,"Miami-Fort Lauderdale-Pompano Beach, FL",12,86,121570.908494,...,5.417691e+05,5.420234e+05,5.412671e+05,5.404553e+05,5.398786e+05,5.393062e+05,5.373761e+05,5.348172e+05,5.317041e+05,5.287793e+05
8,978,8,Dallas County,county,TX,TX,"Dallas-Fort Worth-Arlington, TX",48,113,99121.209545,...,3.294551e+05,3.288662e+05,3.280886e+05,3.272391e+05,3.263679e+05,3.255139e+05,3.239039e+05,3.217117e+05,3.192053e+05,3.168969e+05
9,2832,9,Riverside County,county,CA,CA,"Riverside-San Bernardino-Ontario, CA",6,65,153926.784300,...,6.229825e+05,6.236009e+05,6.243038e+05,6.248867e+05,6.246657e+05,6.237139e+05,6.213239e+05,6.194034e+05,6.170142e+05,6.146041e+05


In [225]:
price_index_df.isna().any().sum()
#price_index_df.dtypes

289

In [226]:
#should probably reshape df so we can merge?

price_df_long = price_index_df.melt(
    id_vars=['RegionID', 'SizeRank', 'RegionName', 'RegionType',
             'StateName', 'State', 'Metro', 
             'StateCodeFIPS', 'MunicipalCodeFIPS'],
    var_name="date",
    value_name="zhvi"
)


price_df_long["date"] = pd.to_datetime(price_df_long["date"])

# keep only year-end values?
price_df_long = price_df_long[price_df_long["date"].dt.month == 12]

# Compute YoY growth within each county
price_df_long["yoy_growth"] = price_df_long.groupby("RegionID")["zhvi"].pct_change()


C:\Users\wesle\AppData\Local\Temp\ipykernel_11904\2795495557.py:18: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  price_df_long["yoy_growth"] = price_df_long.groupby("RegionID")["zhvi"].pct_change()


In [227]:
# 1) Ensure numeric
price_df_long["zhvi"] = pd.to_numeric(price_df_long["zhvi"], errors="coerce")

# 2) Sort within county
price_df_long = price_df_long.sort_values(["RegionID", "date"])

#December-only annual series
annual = price_df_long[price_df_long["date"].dt.month == 12].copy()
annual["yoy_growth"] = (
    annual.groupby("RegionID")["zhvi"].pct_change(fill_method=None)
)

#  If some counties are missing December, take the last month in each year:
tmp = price_df_long.assign(year=price_df_long["date"].dt.year)
annual = (
    tmp.sort_values("date")
       .groupby(["RegionID","RegionName","StateCodeFIPS", "MunicipalCodeFIPS","year",], as_index=False)
       .agg(zhvi=("zhvi","last"))
)
annual["yoy_growth"] = (
    annual.groupby(["RegionID","RegionName","StateCodeFIPS", "MunicipalCodeFIPS"])["zhvi"].pct_change(fill_method=None)
)
annual.head(50)

,RegionID,RegionName,StateCodeFIPS,MunicipalCodeFIPS,year,zhvi,yoy_growth
0,66,Ada County,16,1,2000,NaN,NaN
1,66,Ada County,16,1,2001,154699.183734,NaN
2,66,Ada County,16,1,2002,156751.683350,0.013268
3,66,Ada County,16,1,2003,160114.118314,0.021451
4,66,Ada County,16,1,2004,164041.195545,0.024527
5,66,Ada County,16,1,2005,188176.253463,0.147128
6,66,Ada County,16,1,2006,233554.361847,0.241147
7,66,Ada County,16,1,2007,235465.398581,0.008182
8,66,Ada County,16,1,2008,212471.922535,-0.097651
9,66,Ada County,16,1,2009,184708.882072,-0.130667


In [228]:
df_2000=pd.read_csv(r'C:\Users\wesle\Downloads\2000-2010.csv',encoding="cp1252", engine="python")
df_2010=pd.read_csv(r"C:\Users\wesle\Downloads\2010-2020.csv",encoding="cp1252", engine="python")
df_2020_raw=pd.read_csv(r"C:\Users\wesle\Downloads\co-est2024-alldata.csv",encoding="cp1252", engine="python")

In [229]:
print(f'here are the 2000-2010 columns: {df_2000.columns}')
print(f'here are the 2010-2020 columns: {df_2010.columns}')
print(f'here are the 2020-2024 columns: {df_2020_raw.columns}')

here are the 2000-2010 columns: Index(['SUMLEV', 'STATE', 'COUNTY', 'STNAME', 'CTYNAME', 'SEX', 'AGEGRP',
       'ESTIMATESBASE2000', 'POPESTIMATE2000', 'POPESTIMATE2001',
       'POPESTIMATE2002', 'POPESTIMATE2003', 'POPESTIMATE2004',
       'POPESTIMATE2005', 'POPESTIMATE2006', 'POPESTIMATE2007',
       'POPESTIMATE2008', 'POPESTIMATE2009', 'CENSUS2010POP',
       'POPESTIMATE2010'],
      dtype='object')
here are the 2010-2020 columns: Index(['SUMLEV', 'REGION', 'DIVISION', 'STATE', 'COUNTY', 'STNAME', 'CTYNAME',
       'CENSUS2010POP', 'ESTIMATESBASE2010', 'POPESTIMATE2010',
       ...
       'RNETMIG2011', 'RNETMIG2012', 'RNETMIG2013', 'RNETMIG2014',
       'RNETMIG2015', 'RNETMIG2016', 'RNETMIG2017', 'RNETMIG2018',
       'RNETMIG2019', 'RNETMIG2020'],
      dtype='object', length=180)
here are the 2020-2024 columns: Index(['SUMLEV', 'REGION', 'DIVISION', 'STATE', 'COUNTY', 'STNAME', 'CTYNAME',
       'ESTIMATESBASE2020', 'POPESTIMATE2020', 'POPESTIMATE2021',
       'POPESTIMATE2

In [230]:
#data pipeline before merging, this will combine vertically and add a growth column
# Add YoY growth within each county; only compute when years are consecutive
def _add_yoy(df):
    df = df.sort_values(["fips", "year"]).copy()
    prev_pop = df.groupby("fips")["population"].shift(1)
    prev_year = df.groupby("fips")["year"].shift(1)
    consecutive = (df["year"] - prev_year == 1) & prev_pop.notna()

    df["pop_yoy"] = pd.NA
    df["pop_yoy_pct"] = pd.NA

    df.loc[consecutive, "pop_yoy"] = df.loc[consecutive, "population"].to_numpy() - prev_pop.loc[consecutive].to_numpy()
    valid_base = consecutive & (prev_pop > 0)
    df.loc[valid_base, "pop_yoy_pct"] = (df.loc[valid_base, "population"].to_numpy() / prev_pop.loc[valid_base].to_numpy()) - 1.0


    return df

# Tidy function: county-level totals in long form + YoY within-file
def tidy_county_pop(df):
    df = df.copy()

    # Keep county rows only (050)
    if "SUMLEV" not in df.columns:
        raise ValueError("Expected SUMLEV column not found.")
    df["SUMLEV"] = df["SUMLEV"].astype(str).str.zfill(3)
    df = df.loc[df["SUMLEV"] == "050"]

    # If sex/age breakdown exists, keep totals only (0 = both sexes / all ages)
    if "SEX" in df.columns:
        df = df[df["SEX"].astype(int) == 0]
    if "AGEGRP" in df.columns:
        df = df[df["AGEGRP"].astype(int) == 0]

    # Population estimate columns
    pop_cols = [c for c in df.columns if c.startswith("POPESTIMATE")]
    if not pop_cols:
        raise ValueError("No POPESTIMATE* columns found in DataFrame.")

    # Melt to long format
    id_cols = [c for c in ["STATE", "COUNTY", "STNAME", "CTYNAME"] if c in df.columns]
    long = df[id_cols + pop_cols].melt(
        id_vars=id_cols,
        value_vars=pop_cols,
        var_name="var",
        value_name="population"
    )

    # Extract year
    long["year"] = long["var"].str.extract(r"(\d{4})").astype(int)
    long = long.drop(columns=["var"])

    # Build 5-digit FIPS
    long["state_fips"] = long["STATE"].astype("Int64").astype(str).str.zfill(2)
    long["county_fips3"] = long["COUNTY"].astype("Int64").astype(str).str.zfill(3)
    long["fips"] = long["state_fips"] + long["county_fips3"]

    # Standard names
    long = long.rename(columns={"STNAME": "state", "CTYNAME": "county"})
    long["population"] = pd.to_numeric(long["population"], errors="coerce")
    long = long.dropna(subset=["population"])

    # Keep final columns
    keep = ["fips", "year", "population"]
    if "state" in long.columns: keep.append("state")
    if "county" in long.columns: keep.append("county")
    long = long[keep]

    # Add YoY within this file (useful on its own; will be recomputed after concat)
    long = _add_yoy(long)
    return long


# Tidy to county-year totals 
pop_2000_2010 = tidy_county_pop(df_2000)      # years ~ 2000..2010
pop_2010_2020 = tidy_county_pop(df_2010)      # years ~ 2010..2020
pop_2020_2024 = tidy_county_pop(df_2020_raw)  # years ~ 2020..2024

# Combine on overlaps 
combined = pd.concat([pop_2000_2010, pop_2010_2020, pop_2020_2024], ignore_index=True)
combined = (combined
            .sort_values(["fips", "year"])           
            .drop_duplicates(subset=["fips", "year"], keep="last")
            .reset_index(drop=True))

# Recompute YoY across vintages so 2010 and 2020 transitions are handled cleanly
county_pop = _add_yoy(combined)

print(county_pop.head(10))
print(county_pop.tail(10))
print(county_pop.shape)

    fips  year  population    state          county pop_yoy pop_yoy_pct
0  01001  2000     44021.0  Alabama  Autauga County    <NA>        <NA>
1  01001  2001     44889.0  Alabama  Autauga County   868.0    0.019718
2  01001  2002     45909.0  Alabama  Autauga County  1020.0    0.022723
3  01001  2003     46800.0  Alabama  Autauga County   891.0    0.019408
4  01001  2004     48366.0  Alabama  Autauga County  1566.0    0.033462
5  01001  2005     49676.0  Alabama  Autauga County  1310.0    0.027085
6  01001  2006     51328.0  Alabama  Autauga County  1652.0    0.033255
7  01001  2007     52405.0  Alabama  Autauga County  1077.0    0.020983
8  01001  2008     53277.0  Alabama  Autauga County   872.0     0.01664
9  01001  2009     54135.0  Alabama  Autauga County   858.0    0.016105
        fips  year  population    state         county pop_yoy pop_yoy_pct
78582  56045  2015      7202.0  Wyoming  Weston County    68.0    0.009532
78583  56045  2016      7228.0  Wyoming  Weston County    

In [231]:
# Zero pad to correct widths
annual["state_fips"]  = annual["StateCodeFIPS"].astype(str).str.zfill(2)
annual["county_fips"] = annual["MunicipalCodeFIPS"].astype(str).str.zfill(3)

# Combine into full 5-digit county FIPS
annual["fips"] = annual["state_fips"] + annual["county_fips"]


print(annual[["RegionName","StateCodeFIPS","MunicipalCodeFIPS","fips"]].head())



   RegionName  StateCodeFIPS  MunicipalCodeFIPS   fips
0  Ada County             16                  1  16001
1  Ada County             16                  1  16001
2  Ada County             16                  1  16001
3  Ada County             16                  1  16001
4  Ada County             16                  1  16001


In [232]:
# Ensure types
annual["fips"] = annual["fips"].astype(str).str.zfill(5)
annual["year"] = annual["year"].astype(int)

county_pop["fips"] = county_pop["fips"].astype(str).str.zfill(5)
county_pop["year"] = county_pop["year"].astype(int)

# In case the pop file has multiple rows per (fips,year), keep the last 
county_pop = (
    county_pop
    .sort_values(["fips","year"])
    .drop_duplicates(subset=["fips","year"], keep="last")
)
assert not annual[["fips","year"]].duplicated().any(), "Zillow has duplicate fips-year rows"
assert not county_pop[["fips","year"]].duplicated().any(), "Population has duplicate fips-year rows"


In [233]:
panel = annual.merge(
    county_pop,
    how="inner",                
    on=["fips","year"],
    validate="many_to_one"      
)


In [234]:
panel.head(10)


,RegionID,RegionName,StateCodeFIPS,MunicipalCodeFIPS,year,zhvi,yoy_growth,state_fips,county_fips,fips,population,state,county,pop_yoy,pop_yoy_pct
0,66,Ada County,16,1,2000,NaN,NaN,16,001,16001,303328.0,Idaho,Ada County,<NA>,<NA>
1,66,Ada County,16,1,2001,154699.183734,NaN,16,001,16001,313896.0,Idaho,Ada County,10568.0,0.03484
2,66,Ada County,16,1,2002,156751.683350,0.013268,16,001,16001,321616.0,Idaho,Ada County,7720.0,0.024594
3,66,Ada County,16,1,2003,160114.118314,0.021451,16,001,16001,327393.0,Idaho,Ada County,5777.0,0.017962
4,66,Ada County,16,1,2004,164041.195545,0.024527,16,001,16001,334926.0,Idaho,Ada County,7533.0,0.023009
5,66,Ada County,16,1,2005,188176.253463,0.147128,16,001,16001,348755.0,Idaho,Ada County,13829.0,0.04129
6,66,Ada County,16,1,2006,233554.361847,0.241147,16,001,16001,363498.0,Idaho,Ada County,14743.0,0.042273
7,66,Ada County,16,1,2007,235465.398581,0.008182,16,001,16001,375368.0,Idaho,Ada County,11870.0,0.032655
8,66,Ada County,16,1,2008,212471.922535,-0.097651,16,001,16001,382618.0,Idaho,Ada County,7250.0,0.019314
9,66,Ada County,16,1,2009,184708.882072,-0.130667,16,001,16001,388577.0,Idaho,Ada County,5959.0,0.015574


In [235]:
# Drop all rows where yoy_growth is NaN
df1 = panel.dropna(subset=["yoy_growth","pop_yoy","pop_yoy_pct"])


In [236]:
df1.head(10)

,RegionID,RegionName,StateCodeFIPS,MunicipalCodeFIPS,year,zhvi,yoy_growth,state_fips,county_fips,fips,population,state,county,pop_yoy,pop_yoy_pct
2,66,Ada County,16,1,2002,156751.683350,0.013268,16,001,16001,321616.0,Idaho,Ada County,7720.0,0.024594
3,66,Ada County,16,1,2003,160114.118314,0.021451,16,001,16001,327393.0,Idaho,Ada County,5777.0,0.017962
4,66,Ada County,16,1,2004,164041.195545,0.024527,16,001,16001,334926.0,Idaho,Ada County,7533.0,0.023009
5,66,Ada County,16,1,2005,188176.253463,0.147128,16,001,16001,348755.0,Idaho,Ada County,13829.0,0.04129
6,66,Ada County,16,1,2006,233554.361847,0.241147,16,001,16001,363498.0,Idaho,Ada County,14743.0,0.042273
7,66,Ada County,16,1,2007,235465.398581,0.008182,16,001,16001,375368.0,Idaho,Ada County,11870.0,0.032655
8,66,Ada County,16,1,2008,212471.922535,-0.097651,16,001,16001,382618.0,Idaho,Ada County,7250.0,0.019314
9,66,Ada County,16,1,2009,184708.882072,-0.130667,16,001,16001,388577.0,Idaho,Ada County,5959.0,0.015574
10,66,Ada County,16,1,2010,164926.770937,-0.107099,16,001,16001,393369.0,Idaho,Ada County,4792.0,0.012332
11,66,Ada County,16,1,2011,156138.325006,-0.053287,16,001,16001,401352.0,Idaho,Ada County,7983.0,0.020294


In [237]:
#df1 = df1.drop(columns="county")
df1 = df1.rename(columns={"zhvi": "zillow_index"})
df1.head()

,RegionID,RegionName,StateCodeFIPS,MunicipalCodeFIPS,year,zillow_index,yoy_growth,state_fips,county_fips,fips,population,state,county,pop_yoy,pop_yoy_pct
2,66,Ada County,16,1,2002,156751.683350,0.013268,16,001,16001,321616.0,Idaho,Ada County,7720.0,0.024594
3,66,Ada County,16,1,2003,160114.118314,0.021451,16,001,16001,327393.0,Idaho,Ada County,5777.0,0.017962
4,66,Ada County,16,1,2004,164041.195545,0.024527,16,001,16001,334926.0,Idaho,Ada County,7533.0,0.023009
5,66,Ada County,16,1,2005,188176.253463,0.147128,16,001,16001,348755.0,Idaho,Ada County,13829.0,0.04129
6,66,Ada County,16,1,2006,233554.361847,0.241147,16,001,16001,363498.0,Idaho,Ada County,14743.0,0.042273


In [238]:
df1.shape

(51303, 15)

In [239]:
import io, time, sys, requests, pandas as pd
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter

def make_session():
    s = requests.Session()
    s.headers.update({
        "User-Agent": "Mozilla/5.0 (compatible; county-employment-loader/1.0; youremail@example.com)",
        "Referer": "https://www.bls.gov/lau/tables.htm",
        "Accept": "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet,application/octet-stream,*/*"
    })
    retries = Retry(total=5, backoff_factor=0.5, status_forcelist=[403, 429, 500, 502, 503, 504], raise_on_status=False)
    s.mount("https://", HTTPAdapter(max_retries=retries))
    return s

def read_laus_county_excel(session, url, year):
    r = session.get(url, timeout=60, allow_redirects=True)
    if r.status_code == 403:
        raise RuntimeError(f"403 Forbidden (BLS blocked): {url}")
    r.raise_for_status()
    xl = pd.ExcelFile(io.BytesIO(r.content))
    raw = xl.parse(xl.sheet_names[0], header=None, dtype=str)
    # find header row that contains "State FIPS Code"
    header_row = None
    for i in range(min(30, len(raw))):
        if raw.iloc[i].fillna("").astype(str).str.strip().str.lower().eq("state fips code").any():
            header_row = i
            break
    if header_row is None:
        header_row = 3
    df = xl.parse(xl.sheet_names[0], header=header_row)
    df.columns = [str(c).strip() for c in df.columns]

    # normalize columns
    cmap = {}
    for c in df.columns:
        lc = c.lower()
        if "state" in lc and "fips" in lc: cmap[c] = "state_fips"
        elif "county" in lc and "fips" in lc: cmap[c] = "county_fips"
        elif lc.startswith("area name"): cmap[c] = "area_name"
        elif lc.startswith("labor force"): cmap[c] = "labor_force"
        elif lc.startswith("employ"): cmap[c] = "employment"
        elif lc.startswith("unemploy") and "rate" not in lc: cmap[c] = "unemployment"
        elif "rate" in lc: cmap[c] = "unemployment_rate"
    df = df.rename(columns=cmap)

    df = df[df["state_fips"].notna() & df["county_fips"].notna()].copy()
    df["state_fips"] = df["state_fips"].astype(str).str.extract(r"(\d+)")[0].str.zfill(2)
    df["county_fips"] = df["county_fips"].astype(str).str.extract(r"(\d+)")[0].str.zfill(3)
    df["fips"] = df["state_fips"] + df["county_fips"]
    df["year"] = int(year)

    for col in ["labor_force","employment","unemployment","unemployment_rate"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col].astype(str).str.replace(",", ""), errors="coerce")

    keep = [c for c in ["fips","year","area_name","labor_force","employment","unemployment","unemployment_rate"] if c in df.columns]
    return df[keep].reset_index(drop=True)

def load_laus_county_annual(start=2000, end=2024):
    sess = make_session()
    out = []
    for y in range(start, end+1):
        url = f"https://www.bls.gov/lau/laucnty{y%100:02d}.xlsx"  # official pattern
        try:
            dfy = read_laus_county_excel(sess, url, y)
            out.append(dfy)
            print(f"Loaded {y}: {len(dfy):,} rows")
            time.sleep(0.4)  # be polite
        except Exception as e:
            print(f"Skipping {y} ({url}) due to: {e}", file=sys.stderr)
    if not out:
        raise RuntimeError("No LAUS county files loaded. Check network or headers.")
    return pd.concat(out, ignore_index=True)

laus = load_laus_county_annual(2000, 2024)


Loaded 2000: 3,218 rows
Loaded 2001: 3,218 rows
Loaded 2002: 3,218 rows
Loaded 2003: 3,218 rows
Loaded 2004: 3,218 rows
Loaded 2005: 3,218 rows
Loaded 2006: 3,218 rows
Loaded 2007: 3,218 rows
Loaded 2008: 3,218 rows
Loaded 2009: 3,218 rows
Loaded 2010: 3,220 rows
Loaded 2011: 3,220 rows
Loaded 2012: 3,220 rows
Loaded 2013: 3,220 rows
Loaded 2014: 3,220 rows
Loaded 2015: 3,220 rows
Loaded 2016: 3,220 rows
Loaded 2017: 3,220 rows
Loaded 2018: 3,220 rows
Loaded 2019: 3,220 rows
Loaded 2020: 3,221 rows
Loaded 2021: 3,221 rows
Loaded 2022: 3,221 rows
Loaded 2023: 3,221 rows
Loaded 2024: 3,221 rows


In [240]:
#merge with panel
df2 = df1.merge(
    laus[["fips","year","employment","unemployment_rate","labor_force"]],
    on=["fips","year"], how="left"
)


In [241]:
df2.head()

,RegionID,RegionName,StateCodeFIPS,MunicipalCodeFIPS,year,zillow_index,yoy_growth,state_fips,county_fips,fips,population,state,county,pop_yoy,pop_yoy_pct,employment,unemployment_rate,labor_force
0,66,Ada County,16,1,2002,156751.683350,0.013268,16,001,16001,321616.0,Idaho,Ada County,7720.0,0.024594,167608.0,4.8,176090.0
1,66,Ada County,16,1,2003,160114.118314,0.021451,16,001,16001,327393.0,Idaho,Ada County,5777.0,0.017962,169008.0,4.8,177601.0
2,66,Ada County,16,1,2004,164041.195545,0.024527,16,001,16001,334926.0,Idaho,Ada County,7533.0,0.023009,171194.0,4.1,178421.0
3,66,Ada County,16,1,2005,188176.253463,0.147128,16,001,16001,348755.0,Idaho,Ada County,13829.0,0.04129,180221.0,3.3,186359.0
4,66,Ada County,16,1,2006,233554.361847,0.241147,16,001,16001,363498.0,Idaho,Ada County,14743.0,0.042273,188004.0,2.6,193079.0


In [242]:
df2.shape

(51303, 18)

In [243]:
import pandas as pd
import requests
import zipfile
import io

def fix_encoding_issues():
    """
    Fix the encoding issues with education and population data
    """
    print("Fixing encoding issues...")
    
    # Try different encodings for the problematic files
    files_to_fix = {
        'education': 'https://ers.usda.gov/sites/default/files/_laserfiche/DataFiles/48747/Education2023.xlsx',
        'population': 'https://ers.usda.gov/sites/default/files/_laserfiche/DataFiles/48747/PopulationEstimates.xlsx'
    }
    
    for name, url in files_to_fix.items():
        try:
            # Try Excel format instead of CSV
            df = pd.read_excel(url)
            filename = f"county_{name}_data.csv"
            df.to_csv(filename, index=False)
            print(f"Fixed {name}: {len(df):,} records -> {filename}")
        except Exception as e:
            print(f"Still error with {name}: {e}")

def download_hud_fair_market_rents(years=[2023, 2022, 2021]):
    """
    Download HUD Fair Market Rents - county-level housing costs
    This is perfect for housing prediction!
    """
    print("Downloading HUD Fair Market Rents (county-level housing costs)...")
    
    all_data = []
    
    for year in years:
        url = f"https://www.huduser.gov/portal/datasets/fmr/fmr{year}/FY{year}_FMRs.xlsx"
        try:
            df = pd.read_excel(url)
            df['year'] = year
            all_data.append(df)
            print(f"  {year}: {len(df):,} records")
        except Exception as e:
            print(f"  Error with {year}: {e}")
    
    if all_data:
        combined = pd.concat(all_data, ignore_index=True)
        combined.to_csv('county_fair_market_rents.csv', index=False)
        print(f"Saved combined FMR data: {len(combined):,} records -> county_fair_market_rents.csv")
        return combined
    
    return pd.DataFrame()

def download_bls_qcew_actual(year=2023):
    """
    Actually download and process BLS QCEW data for one year
    """
    print(f"Downloading BLS QCEW wage data for {year}...")
    
    url = f"https://www.bls.gov/cew/downloadable-data-files/annual/csv/{year}_annual_by_area.zip"
    
    try:
        response = requests.get(url, timeout=60)
        response.raise_for_status()
        
        with zipfile.ZipFile(io.BytesIO(response.content)) as zip_file:
            # Find county-level file
            csv_files = [f for f in zip_file.namelist() if 'county' in f.lower() and f.endswith('.csv')]
            
            if not csv_files:
                # Try any CSV file
                csv_files = [f for f in zip_file.namelist() if f.endswith('.csv')]
            
            if csv_files:
                with zip_file.open(csv_files[0]) as csv_file:
                    df = pd.read_csv(csv_file, dtype=str)
                    
                    # Filter for county-level data (area_fips length = 5)
                    if 'area_fips' in df.columns:
                        county_data = df[df['area_fips'].str.len() == 5].copy()
                        
                        # Keep important columns
                        important_cols = ['area_fips', 'avg_wkly_wage', 'total_qtrly_wages', 'qtrly_estabs']
                        available_cols = [col for col in important_cols if col in county_data.columns]
                        
                        if available_cols:
                            result = county_data[available_cols].copy()
                            result['year'] = year
                            
                            filename = f'county_wages_{year}.csv'
                            result.to_csv(filename, index=False)
                            
                            print(f"Success: {len(result):,} counties -> {filename}")
                            return result
            
            print("No suitable CSV files found in zip")
            
    except Exception as e:
        print(f"Error downloading QCEW: {e}")
    
    return pd.DataFrame()

def analyze_what_you_have():
    """
    Analyze the data you successfully downloaded
    """
    print("ANALYZING YOUR CURRENT DATA")
    print("="*40)
    
    files_to_check = [
        'county_unemployment_data.csv',
        'county_poverty_data.csv', 
        'state_gdp_data.csv'
    ]
    
    for filename in files_to_check:
        try:
            df = pd.read_csv(filename)
            print(f"\n{filename}:")
            print(f"  Shape: {df.shape}")
            print(f"  Columns: {list(df.columns)[:5]}...")  # First 5 columns
            
            # Check for FIPS codes
            fips_cols = [col for col in df.columns if 'fips' in col.lower()]
            if fips_cols:
                print(f"  FIPS columns: {fips_cols}")
            
            # Check for year columns  
            year_cols = [col for col in df.columns if 'year' in col.lower() or any(str(y) in str(col) for y in range(2000, 2025))]
            if year_cols:
                print(f"  Year columns: {year_cols[:3]}...")
                
        except Exception as e:
            print(f"\nError reading {filename}: {e}")

def create_housing_prediction_dataset():
    """
    Combine your downloaded data into a housing prediction dataset
    """
    print("\nCREATING HOUSING PREDICTION DATASET")
    print("="*40)
    
    try:
        # Load unemployment data
        unemployment = pd.read_csv('county_unemployment_data.csv')
        print(f"Unemployment data: {unemployment.shape}")
        
        # Load poverty data  
        poverty = pd.read_csv('county_poverty_data.csv')
        print(f"Poverty data: {poverty.shape}")
        
        # Find common FIPS column
        unemployment_fips = [col for col in unemployment.columns if 'fips' in col.lower()]
        poverty_fips = [col for col in poverty.columns if 'fips' in col.lower()]
        
        print(f"Unemployment FIPS cols: {unemployment_fips}")
        print(f"Poverty FIPS cols: {poverty_fips}")
        
        # Show sample of each dataset
        print(f"\nUnemployment sample:")
        print(unemployment.head(2))
        
        print(f"\nPoverty sample:")
        print(poverty.head(2))
        
        # Suggest next steps based on what we see
        print(f"\nNEXT STEPS:")
        print(f"1. Identify common FIPS codes for merging")
        print(f"2. Extract time series data (multiple years)")
        print(f"3. Add housing price data as target variable")
        
    except Exception as e:
        print(f"Error: {e}")

def get_next_steps():
    """
    What to do next for housing prediction
    """
    print("\nRECOMMENDED NEXT STEPS")
    print("="*30)
    
    print("\n1. FIX ENCODING ISSUES:")
    print("   Run: fix_encoding_issues()")
    print("   This gets education data (strong housing predictor)")
    
    print("\n2. ADD HOUSING PRICE DATA:")  
    print("   Run: download_hud_fair_market_rents()")
    print("   This gives you target variable for prediction")
    
    print("\n3. ADD WAGE DATA:")
    print("   Run: download_bls_qcew_actual(2023)")  
    print("   This gives you income proxy")
    
    print("\n4. MERGE DATASETS:")
    print("   Combine on FIPS codes")
    print("   Create final prediction dataset")
    
    print(f"\nPRIORITY ORDER:")
    print(f"1. Housing prices (target) - HUD Fair Market Rents")
    print(f"2. Income (strongest predictor) - QCEW wages or fix education")
    print(f"3. Your unemployment + poverty (already have)")

if __name__ == "__main__":
    analyze_what_you_have()
    
    get_next_steps()
    
    print(f"\n" + "="*50)
    print("QUICK FIXES YOU CAN RUN NOW:")
    print("1. fix_encoding_issues()          # Get education data")
    print("2. download_hud_fair_market_rents() # Get housing prices") 
    print("3. download_bls_qcew_actual(2023)   # Get wage data")

ANALYZING YOUR CURRENT DATA

county_unemployment_data.csv:
  Shape: (329726, 5)
  Columns: ['FIPS_Code', 'State', 'Area_Name', 'Attribute', 'Value']...
  FIPS columns: ['FIPS_Code']

county_poverty_data.csv:
  Shape: (79961, 5)
  Columns: ['FIPS_Code', 'Stabr', 'Area_Name', 'Attribute', 'Value']...
  FIPS columns: ['FIPS_Code']

state_gdp_data.csv:
  Shape: (314, 2)
  Columns: ['observation_date', 'GDPC1']...

RECOMMENDED NEXT STEPS

1. FIX ENCODING ISSUES:
   Run: fix_encoding_issues()
   This gets education data (strong housing predictor)

2. ADD HOUSING PRICE DATA:
   Run: download_hud_fair_market_rents()
   This gives you target variable for prediction

3. ADD WAGE DATA:
   Run: download_bls_qcew_actual(2023)
   This gives you income proxy

4. MERGE DATASETS:
   Combine on FIPS codes
   Create final prediction dataset

PRIORITY ORDER:
1. Housing prices (target) - HUD Fair Market Rents
2. Income (strongest predictor) - QCEW wages or fix education
3. Your unemployment + poverty (alre

In [244]:
# merge_unemp_ers_and_poverty_saipe.py
# - ERS unemployment (2000–2023)
# - SAIPE poverty rate (county, multi-year via Census API)
# Produces wide panel with unemployment_rate + poverty_rate for as many years as available.

import re, time, requests
import pandas as pd

USDA_UNEMPLOYMENT_URL = "https://ers.usda.gov/sites/default/files/_laserfiche/DataFiles/48747/Unemployment2023.xlsx"
# We will NOT use the ERS Poverty2023.xlsx (single-year). We'll use SAIPE API instead.

# -----------------------------
# Helpers
# -----------------------------
def normalize_year_series(s: pd.Series) -> pd.Series:
    y = s.astype(str).str.extract(r"((?:19|20)\d{2})")[0]
    return pd.to_numeric(y, errors="coerce").astype("Int64")

def _clean_percent_series(s: pd.Series) -> pd.Series:
    return (
        s.astype(str)
         .str.replace("%", "", regex=False)
         .str.replace(",", "", regex=False)
         .str.extract(r"(-?\d+(?:\.\d+)?)")[0]
         .astype(float)
    )

def _coverage_by_year(df: pd.DataFrame, col: str) -> pd.Series:
    if col not in df.columns: return pd.Series(dtype=float)
    return (df.assign(_has=df[col].notna())
              .groupby("year")["_has"].mean()
              .mul(100).round(1))

def _coalesce_rate(df: pd.DataFrame, base: str) -> pd.DataFrame:
    y, x, t = f"{base}_rate_y", f"{base}_rate_x", f"{base}_rate"
    if y in df.columns: df[t] = df[y]
    elif t in df.columns: df[t] = df[t]
    elif x in df.columns: df[t] = df[x]
    for c in (x, y):
        if c in df.columns: df.drop(columns=c, inplace=True)
    return df


# ERS unemployment (multi-year)

def _read_excel_headerless(url_or_path):
    return pd.read_excel(url_or_path, header=None, dtype=str)

def _find_header_row(df0: pd.DataFrame) -> int:
    tokens = {"fips", "fipstxt", "fips code", "fips_code", "area_name", "stabr", "county_name", "name"}
    for i in range(min(50, len(df0))):
        row = df0.iloc[i].astype(str).str.strip().str.lower().tolist()
        if any(tok in row for tok in tokens): return i
    return 0

def _set_header(df0: pd.DataFrame) -> pd.DataFrame:
    hdr = _find_header_row(df0)
    df = df0.iloc[hdr+1:].reset_index(drop=True)
    cols = df0.iloc[hdr].astype(str).str.strip().tolist()
    cols = [re.sub(r"\s+", " ", c).strip() for c in cols]
    df.columns = cols
    return df

def _extract_fips(df: pd.DataFrame) -> pd.DataFrame:
    cand = [c for c in df.columns if str(c).strip().lower() in {"fips","fipstxt","fips code","fips_code"}]
    if not cand:
        for c in df.columns:
            v = df[c].astype(str).str.replace(r"[^0-9]","",regex=True)
            if v.str.len().between(4,12).mean() > 0.6:
                cand.append(c)
    if not cand: raise ValueError("No FIPS column found after header normalization.")
    f = (df[cand[0]].astype(str).str.replace(r"[^0-9]","",regex=True).str.zfill(5).str[:5])
    out = df.copy()
    out["county_fips"] = f
    out = out[out["county_fips"].str.fullmatch(r"\d{5}")]
    return out

def read_usda_unemployment():
    print("Reading ERS unemployment workbook...")
    df0 = _read_excel_headerless(USDA_UNEMPLOYMENT_URL)
    df = _set_header(df0)
    print("  Columns (first 10):", list(df.columns)[:10])
    return df

def process_unemployment_data(df: pd.DataFrame) -> pd.DataFrame:
    if df is None or df.empty: return pd.DataFrame()
    print("\nProcessing unemployment data...")
    df = _extract_fips(df)

    pairs = []
    for c in df.columns:
        cl = str(c).lower()
        m = re.search(r"(19|20)\d{2}", cl)
        if not m: continue
        y = int(m.group(0))
        if ("unemploy" in cl) and ("rate" in cl):
            pairs.append((c, y))

    if not pairs:
        print("  No unemployment rate year columns found.")
        return pd.DataFrame()

    keep = ["county_fips"] + [c for c,_ in pairs]
    long = df[keep].melt(id_vars=["county_fips"], var_name="col", value_name="unemployment_rate")
    long["year"] = long["col"].map({c:y for c,y in pairs})
    long.drop(columns=["col"], inplace=True)
    long["unemployment_rate"] = _clean_percent_series(long["unemployment_rate"])
    long = long.dropna(subset=["year","unemployment_rate"]).astype({"year": int})
    print(f"  Total unemployment records: {len(long)}")
    return long

# -----------------------------
# SAIPE poverty (multi-year via API)
SAIPE_BASE = "https://api.census.gov/data/timeseries/poverty/saipe"

def _discover_poverty_var() -> str:
    """
    Ask the SAIPE API for its variable catalog and return the best variable
    representing 'Percent of people in poverty, all ages (point estimate)'.
    """
    r = requests.get(f"{SAIPE_BASE}/variables.json", timeout=30)
    r.raise_for_status()
    variables = r.json().get("variables", {})

    # Preference order (handle naming differences across vintages):
    candidates = [
        "SAEPOVRT_ALL",       # percent in poverty (all ages)
        "SAEPOVRT_ALL_PT",    # sometimes the point estimate is explicit
        "SAEPOVRTALL_PT",     # alt spelling in some docs/tools
        "PCTPOVALL",          # old-style alias sometimes used
    ]
    for v in candidates:
        if v in variables:
            return v

    # Last resort: fuzzy search
    for k in variables.keys():
        if re.fullmatch(r"SAEPOVRT[_A-Z]*ALL(_PT)?", k):
            return k
    raise RuntimeError("Couldn't find a SAIPE variable for county poverty rate (all ages).")

def fetch_saipe_poverty(years=range(2000, 2025), sleep_sec=0.3) -> pd.DataFrame:
    """
    Pull county-level poverty rate for all available years in `years`.
    Returns: DataFrame[county_fips, year, poverty_rate]
    """
    var = _discover_poverty_var()
    print(f"Using SAIPE variable: {var}")

    frames = []
    for y in years:
        params = {
            "get": f"NAME,{var},STATE,COUNTY",
            "for": "county:*",
            "time": y
        }
        try:
            r = requests.get(SAIPE_BASE, params=params, timeout=60)
            if r.status_code != 200:
                print(f"  Year {y}: HTTP {r.status_code} — skipping")
                time.sleep(sleep_sec)
                continue
            rows = r.json()
            header, data = rows[0], rows[1:]
            df = pd.DataFrame(data, columns=header)
            # Build county_fips and year
            df["county_fips"] = df["STATE"].str.zfill(2) + df["COUNTY"].str.zfill(3)
            df["year"] = pd.to_numeric(df["time"])
            # numeric poverty rate
            df["poverty_rate"] = pd.to_numeric(df[var], errors="coerce")
            frames.append(df[["county_fips","year","poverty_rate"]])
            print(f"  Year {y}: {len(df)} counties")
        except Exception as e:
            print(f"  Year {y}: error {e} — skipping")
        time.sleep(sleep_sec)  # be nice to the API

    if not frames:
        print("No SAIPE data retrieved.")
        return pd.DataFrame(columns=["county_fips","year","poverty_rate"])

    out = pd.concat(frames, ignore_index=True)
    out = out.sort_values(["county_fips","year"]).reset_index(drop=True)
    print(f"SAIPE collected rows: {len(out)} across years {out['year'].min()}–{out['year'].max()}")
    return out

# -----------------------------
# Merge everything with your housing panel
# -----------------------------
def derive_housing_fips5(housing_df: pd.DataFrame) -> pd.Series:
    cols = {c.lower(): c for c in housing_df.columns}
    if "fips" in cols:
        s = housing_df[cols["fips"]].astype(str).str.replace(r"[^0-9]","",regex=True)
        return s.str.zfill(5).str[-5:]
    state_cands = [k for k in cols if k in {"state_fips","statecodefips","state_code_fips","statecode"}]
    county_cands = [k for k in cols if k in {"county_fips","countycodefips","county_code_fips","countycode"}]
    if state_cands and county_cands:
        s2 = housing_df[cols[state_cands[0]]].astype(str).str.replace(r"[^0-9]","",regex=True).str.zfill(2).str[-2:]
        c3 = housing_df[cols[county_cands[0]]].astype(str).str.replace(r"[^0-9]","",regex=True).str.zfill(3).str[-3:]
        return s2 + c3
    raise ValueError("Need either a 5-digit 'fips' or both 'state_fips' and 'county_fips' in housing_df.")

def merge_unemp_and_saipe(housing_df: pd.DataFrame) -> pd.DataFrame:
    print("MERGING COUNTY DATA WITH HOUSING (ERS Unemployment + SAIPE Poverty)")
    print("="*60)
    print("Housing shape:", housing_df.shape)

    df = housing_df.copy()
    df["county_fips"] = derive_housing_fips5(df)
    df["year"] = normalize_year_series(df["year"]).fillna(df["year"]).astype(int)

    # ERS unemployment
    ers_unemp_wb = read_usda_unemployment()
    unemp_long = process_unemployment_data(ers_unemp_wb)

    # SAIPE poverty
    saipe_long = fetch_saipe_poverty(years=range(2000, 2025))

    # Overlap diagnostics
    kh = set(zip(df["county_fips"], df["year"]))
    ku = set(zip(unemp_long["county_fips"], unemp_long["year"])) if not unemp_long.empty else set()
    kp = set(zip(saipe_long["county_fips"], saipe_long["year"])) if not saipe_long.empty else set()
    print(f"Overlapping keys — unemployment: {len(kh & ku):,}, poverty: {len(kh & kp):,}")

    # Merge
    out = df.merge(unemp_long, on=["county_fips","year"], how="left")
    out = _coalesce_rate(out, "unemployment")

    out = out.merge(saipe_long, on=["county_fips","year"], how="left")  # poverty_rate

    # Coverage
    cov_u = _coverage_by_year(out, "unemployment_rate")
    cov_p = _coverage_by_year(out, "poverty_rate")
    print("\nCoverage by year (unemployment):")
    print(cov_u.sort_index().to_string())
    print("\nCoverage by year (poverty via SAIPE):")
    print(cov_p.sort_index().to_string())

    return out

# --- Run on your df2 panel:
merged_df = merge_unemp_and_saipe(df2)


MERGING COUNTY DATA WITH HOUSING (ERS Unemployment + SAIPE Poverty)
Housing shape: (51303, 18)
Reading ERS unemployment workbook...
  Columns (first 10): ['FIPS_Code', 'State', 'Area_Name', 'Rural_Urban_Continuum_Code_2023', 'Urban_Influence_Code_2013', 'Metro_2023', 'Civilian_labor_force_2000', 'Employed_2000', 'Unemployed_2000', 'Unemployment_rate_2000']

Processing unemployment data...
  Total unemployment records: 78419
Using SAIPE variable: SAEPOVRTALL_PT
  Year 2000: 3140 counties
  Year 2001: 3141 counties
  Year 2002: 3141 counties
  Year 2003: 3141 counties
  Year 2004: 3141 counties
  Year 2005: 3141 counties
  Year 2006: 3141 counties
  Year 2007: 3141 counties
  Year 2008: 3142 counties
  Year 2009: 3143 counties
  Year 2010: 3143 counties
  Year 2011: 3143 counties
  Year 2012: 3143 counties
  Year 2013: 3143 counties
  Year 2014: 3142 counties
  Year 2015: 3142 counties
  Year 2016: 3142 counties
  Year 2017: 3142 counties
  Year 2018: 3142 counties
  Year 2019: 3142 coun

In [245]:
merged_df.head(50)

,RegionID,RegionName,StateCodeFIPS,MunicipalCodeFIPS,year,zillow_index,yoy_growth,state_fips,county_fips,fips,population,state,county,pop_yoy,pop_yoy_pct,employment,labor_force,unemployment_rate,poverty_rate
0,66,Ada County,16,1,2002,156751.683350,0.013268,16,16001,16001,321616.0,Idaho,Ada County,7720.0,0.024594,167608.0,176090.0,4.8,8.6
1,66,Ada County,16,1,2003,160114.118314,0.021451,16,16001,16001,327393.0,Idaho,Ada County,5777.0,0.017962,169008.0,177601.0,4.8,9.3
2,66,Ada County,16,1,2004,164041.195545,0.024527,16,16001,16001,334926.0,Idaho,Ada County,7533.0,0.023009,171194.0,178421.0,4.0,9.1
3,66,Ada County,16,1,2005,188176.253463,0.147128,16,16001,16001,348755.0,Idaho,Ada County,13829.0,0.04129,180221.0,186359.0,3.3,9.4
4,66,Ada County,16,1,2006,233554.361847,0.241147,16,16001,16001,363498.0,Idaho,Ada County,14743.0,0.042273,188004.0,193079.0,2.7,8.3
5,66,Ada County,16,1,2007,235465.398581,0.008182,16,16001,16001,375368.0,Idaho,Ada County,11870.0,0.032655,191856.0,196955.0,2.7,7.9
6,66,Ada County,16,1,2008,212471.922535,-0.097651,16,16001,16001,382618.0,Idaho,Ada County,7250.0,0.019314,188641.0,198070.0,4.8,9.0
7,66,Ada County,16,1,2009,184708.882072,-0.130667,16,16001,16001,388577.0,Idaho,Ada County,5959.0,0.015574,180950.0,199261.0,8.9,11.8
8,66,Ada County,16,1,2010,164926.770937,-0.107099,16,16001,16001,393369.0,Idaho,Ada County,4792.0,0.012332,181295.0,198087.0,8.1,13.9
9,66,Ada County,16,1,2011,156138.325006,-0.053287,16,16001,16001,401352.0,Idaho,Ada County,7983.0,0.020294,185978.0,200954.0,7.2,13.4


In [262]:
# === model_prep_ready_v2_transform.py ===
# Build a leak-safe, model-ready table from your merged_df (ERS+SAIPE already merged).

from __future__ import annotations
import numpy as np
import pandas as pd
from pathlib import Path
import datetime

# ----------------- small helpers -----------------
def _norm_year(s: pd.Series) -> pd.Series:
    y = s.astype(str).str.extract(r'((?:19|20)\d{2})')[0]
    return pd.to_numeric(y, errors="coerce")

def _coverage_by_year(df: pd.DataFrame, col: str) -> pd.Series:
    if col not in df.columns: 
        return pd.Series(dtype=float)
    return (df.assign(_has=df[col].notna())
              .groupby("year")["_has"].mean()
              .mul(100).round(1))

def _safe_rate(numer, denom, factor=100_000):
    return np.where((denom>0) & np.isfinite(denom), (numer/denom)*factor, np.nan)

def _maybe_read(path_or_df, dtype=None):
    if path_or_df is None:
        return None
    if isinstance(path_or_df, pd.DataFrame):
        return path_or_df.copy()
    p = Path(path_or_df)
    if not p.exists():
        print(f"(skip) not found: {p}")
        return None
    if p.suffix.lower() == ".parquet":
        return pd.read_parquet(p)
    return pd.read_csv(p, dtype=dtype)

# ----------------- core feature builders -----------------
def build_unemp_features(df: pd.DataFrame) -> pd.DataFrame:
    """As-of-t unemployment features. If t is missing (e.g., 2024), use lag1."""
    df = df.sort_values(["county_fips","year"]).copy()
    if "unemployment_rate" not in df.columns:
        df["unemployment_rate"] = np.nan
    df["unemployment_rate"] = pd.to_numeric(df["unemployment_rate"], errors="coerce")

    grp = df.groupby("county_fips")["unemployment_rate"]
    df["unemp_lag1"] = grp.shift(1)
    df["unemp_ma3"]  = grp.transform(lambda s: s.rolling(3, min_periods=1).mean())
    df["unemp_chg_1y"] = df["unemployment_rate"] - df["unemp_lag1"]
    df["unemp_asof"] = np.where(df["unemployment_rate"].notna(), df["unemployment_rate"], df["unemp_lag1"])
    return df

def build_poverty_features(df: pd.DataFrame) -> pd.DataFrame:
    """As-of-t poverty features from SAIPE. If 2024/25 missing, use lag1."""
    df = df.sort_values(["county_fips","year"]).copy()
    if "poverty_rate" not in df.columns:
        df["poverty_rate"] = np.nan
    df["poverty_rate"] = pd.to_numeric(df["poverty_rate"], errors="coerce")

    grp = df.groupby("county_fips")["poverty_rate"]
    df["pov_lag1"] = grp.shift(1)
    df["pov_ma3"]  = grp.transform(lambda s: s.rolling(3, min_periods=1).mean())
    df["pov_chg_1y"] = df["poverty_rate"] - df["pov_lag1"]
    df["pov_asof"] = np.where(df["poverty_rate"].notna(), df["poverty_rate"], df["pov_lag1"])
    return df

def merge_crime(df_panel: pd.DataFrame, crime: str | pd.DataFrame | None = None) -> pd.DataFrame:
    """
    Optional: merge county-year crime.
    Accepts either per-100k columns or raw counts + population.
    """
    if crime is None:
        print("Crime: (skip)")
        return df_panel

    print("Merging crime…")
    cr = _maybe_read(crime, dtype={"county_fips":"string"})
    if cr is None:
        print("  Crime: not found, skipping.")
        return df_panel

    cr = cr.copy()
    cr["county_fips"] = cr["county_fips"].astype(str).str.zfill(5)
    cr["year"] = _norm_year(cr["year"]).fillna(cr["year"]).astype(int)

    # Build rates if only counts provided
    if "violent_crime_per_100k" not in cr.columns and {"violent_crime","population"}.issubset(cr.columns):
        cr["violent_crime_per_100k"] = _safe_rate(cr["violent_crime"], cr["population"])
    if "property_crime_per_100k" not in cr.columns and {"property_crime","population"}.issubset(cr.columns):
        cr["property_crime_per_100k"] = _safe_rate(cr["property_crime"], cr["population"])

    keep = ["county_fips","year"] + [c for c in ["violent_crime_per_100k","property_crime_per_100k"] if c in cr.columns]
    cr = cr[keep].drop_duplicates()

    out = (df_panel.merge(cr, on=["county_fips","year"], how="left")
                  .sort_values(["county_fips","year"]))

    # Use transform (flat index) for lags/changes
    for c in ["violent_crime_per_100k","property_crime_per_100k"]:
        if c in out.columns:
            g = out.groupby("county_fips")[c]
            out[c+"_lag1"]   = g.shift(1)
            out[c+"_pchg_3y"] = g.transform(lambda s: s.pct_change(3))
            out[c+"_pchg_5y"] = g.transform(lambda s: s.pct_change(5))

            cov = _coverage_by_year(out, c)
            if not cov.empty:
                print(f"  Coverage ({c}) last 5 years:\n{cov.sort_index().tail(5).to_string()}")

    return out

def merge_climate(df_panel: pd.DataFrame, climate: str | pd.DataFrame | None = None) -> pd.DataFrame:
    """Optional: merge climate risk (static or year-indexed)."""
    if climate is None:
        print("Climate: (skip)")
        return df_panel

    print("Merging climate…")
    cl = _maybe_read(climate, dtype={"county_fips":"string"})
    if cl is None:
        print("  Climate: not found, skipping.")
        return df_panel

    cl = cl.copy()
    cl["county_fips"] = cl["county_fips"].astype(str).str.zfill(5)
    if "year" in cl.columns:
        cl["year"] = _norm_year(cl["year"]).fillna(cl["year"]).astype(int)
        return df_panel.merge(cl, on=["county_fips","year"], how="left")
    else:
        cl = cl.drop_duplicates("county_fips")
        return df_panel.merge(cl, on="county_fips", how="left")

def make_target(df: pd.DataFrame, hpi_cols=("zillow_index","zhvi","hpi","fhfa_hpi")) -> tuple[pd.DataFrame,str]:
    """y = log(HPI_{t+1}) - log(HPI_t)"""
    df = df.sort_values(["county_fips","year"]).copy()
    hpi_col = next((c for c in hpi_cols if c in df.columns), None)
    if hpi_col is None:
        raise ValueError(f"No HPI column found. Expected one of {hpi_cols}.")
    df["hpi_lead"] = df.groupby("county_fips")[hpi_col].shift(-1)
    df["y_growth_next"] = np.log(df["hpi_lead"]) - np.log(df[hpi_col])
    return df, hpi_col

def add_crime_and_climate(merged_df, crime=None, climate=None, engineer_crime=True):
    """
    Return `merged_df` augmented with crime and climate features.

    Parameters
    ----------
    merged_df : pd.DataFrame
        Must contain `county_fips` (2+3 digits) and `year` (int or parseable).
    crime : (pd.DataFrame | str | pathlib.Path) optional
        County-year crime data. Accepts either per-100k columns or raw counts.
        Expected any of:
          - violent_crime_per_100k / property_crime_per_100k
          - OR violent_crime, property_crime, population (to compute per-100k)
    climate : (pd.DataFrame | str | pathlib.Path) optional
        County climate features. If it has a `year` column, merge by (fips, year);
        otherwise treat as static per county and broadcast to all years.
    engineer_crime : bool
        If True, adds lag and multi-year % change features for crime.

    Returns
    -------
    pd.DataFrame
        `merged_df` with crime and climate columns (and crime lags/trends if requested).
    """
    import numpy as np
    import pandas as pd
    from pathlib import Path

    # ---------- tiny helpers (kept local so it's truly drop-in) ----------
    def _norm_year(s: pd.Series) -> pd.Series:
        y = pd.to_numeric(pd.Series(s).astype(str).str.extract(r'((?:19|20)\d{2})')[0], errors="coerce")
        return y

    def _safe_rate(numer, denom, factor=100_000):
        numer = pd.to_numeric(numer, errors="coerce")
        denom = pd.to_numeric(denom, errors="coerce")
        return np.where((denom > 0) & np.isfinite(denom), (numer / denom) * factor, np.nan)

    def _maybe_read(obj, dtype=None):
        if obj is None:
            return None
        if isinstance(obj, pd.DataFrame):
            return obj.copy()
        p = Path(obj)
        if not p.exists():
            raise FileNotFoundError(f"File not found: {p}")
        if p.suffix.lower() == ".parquet":
            return pd.read_parquet(p)
        return pd.read_csv(p, dtype=dtype)

    # ---------- start with a clean key ----------
    df = merged_df.copy()
    df["county_fips"] = df["county_fips"].astype(str).str.replace(r"[^0-9]", "", regex=True).str.zfill(5)
    df["year"] = _norm_year(df["year"]).fillna(df["year"]).astype("Int64")
    df = df.sort_values(["county_fips", "year"])

    # ---------- merge CRIME (if provided) ----------
    cr = _maybe_read(crime, dtype={"county_fips": "string"}) if crime is not None else None
    if cr is not None:
        cr = cr.copy()
        cr["county_fips"] = cr["county_fips"].astype(str).str.replace(r"[^0-9]", "", regex=True).str.zfill(5)
        cr["year"] = _norm_year(cr["year"]).fillna(cr["year"]).astype("Int64")

        # Build rates if only counts provided
        if "violent_crime_per_100k" not in cr.columns and {"violent_crime", "population"}.issubset(cr.columns):
            cr["violent_crime_per_100k"] = _safe_rate(cr["violent_crime"], cr["population"])
        if "property_crime_per_100k" not in cr.columns and {"property_crime", "population"}.issubset(cr.columns):
            cr["property_crime_per_100k"] = _safe_rate(cr["property_crime"], cr["population"])

        keep_cols = ["county_fips", "year"]
        for c in ["violent_crime_per_100k", "property_crime_per_100k"]:
            if c in cr.columns: keep_cols.append(c)
        cr = cr[keep_cols].drop_duplicates()

        df = df.merge(cr, on=["county_fips", "year"], how="left").sort_values(["county_fips", "year"])

        if engineer_crime:
            for c in ["violent_crime_per_100k", "property_crime_per_100k"]:
                if c in df.columns:
                    g = df.groupby("county_fips")[c]
                    df[f"{c}_lag1"]    = g.shift(1)
                    df[f"{c}_pchg_3y"] = g.transform(lambda s: s.pct_change(3))
                    df[f"{c}_pchg_5y"] = g.transform(lambda s: s.pct_change(5))

    # ---------- merge CLIMATE (if provided) ----------
    cl = _maybe_read(climate, dtype={"county_fips": "string"}) if climate is not None else None
    if cl is not None:
        cl = cl.copy()
        cl["county_fips"] = cl["county_fips"].astype(str).str.replace(r"[^0-9]", "", regex=True).str.zfill(5)
        if "year" in cl.columns:
            cl["year"] = _norm_year(cl["year"]).fillna(cl["year"]).astype("Int64")
            df = df.merge(cl, on=["county_fips", "year"], how="left")
        else:
            df = df.merge(cl.drop_duplicates("county_fips"), on="county_fips", how="left")

    return df




merged_with_cc = add_crime_and_climate(merged_df)

merged_with_cc.head()



,RegionID,RegionName,StateCodeFIPS,MunicipalCodeFIPS,year,zillow_index,yoy_growth,state_fips,county_fips,fips,population,state,county,pop_yoy,pop_yoy_pct,employment,labor_force,unemployment_rate,poverty_rate
22622,1524,Autauga County,1,1,2001,128468.753216,0.026346,01,01001,01001,44889.0,Alabama,Autauga County,868.0,0.019718,21220.0,22127.0,4.1,10.8
22623,1524,Autauga County,1,1,2002,131298.995264,0.022031,01,01001,01001,45909.0,Alabama,Autauga County,1020.0,0.022723,21189.0,22252.0,4.8,10.3
22624,1524,Autauga County,1,1,2003,134991.165015,0.028120,01,01001,01001,46800.0,Alabama,Autauga County,891.0,0.019408,21666.0,22804.0,5.0,10.4
22625,1524,Autauga County,1,1,2004,142262.582444,0.053866,01,01001,01001,48366.0,Alabama,Autauga County,1566.0,0.033462,22283.0,23385.0,4.7,11.6
22626,1524,Autauga County,1,1,2005,153114.666686,0.076282,01,01001,01001,49676.0,Alabama,Autauga County,1310.0,0.027085,23130.0,24038.0,3.8,10.4


In [264]:
# === make_model_ready_dataset.py ===
# Inputs: a county-year panel with HPI + unemployment + poverty (+ optional crime/climate)
# Output: model_ready_[DATE].parquet / .csv (features + target). No modeling.

import numpy as np
import pandas as pd
from pathlib import Path
import datetime

OUTDIR = Path("data/out")

# ---------- helpers ----------
def _norm_year(s: pd.Series) -> pd.Series:
    y = s.astype(str).str.extract(r'((?:19|20)\d{2})')[0]
    return pd.to_numeric(y, errors="coerce")

def _ensure_keys(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out["county_fips"] = out["county_fips"].astype(str).str.replace(r"[^0-9]","",regex=True).str.zfill(5)
    out["year"] = _norm_year(out["year"]).fillna(out["year"]).astype(int)
    return out.sort_values(["county_fips","year"])

def _detect_hpi_col(df, candidates=("zillow_index","zhvi","hpi","fhfa_hpi")):
    for c in candidates:
        if c in df.columns:
            return c
    raise ValueError(f"No HPI column found. Expected one of {candidates}.")

def _coverage_by_year(df, col):
    if col not in df.columns: return pd.Series(dtype=float)
    return (df.assign(_has=df[col].notna())
              .groupby("year")["_has"].mean()
              .mul(100).round(1))

def _winsorize(s, p_low=0.5, p_high=99.5):
    x = pd.to_numeric(s, errors="coerce")
    lo, hi = np.nanpercentile(x, [p_low, p_high])
    return x.clip(lo, hi)

# ---------- feature builders ----------
def add_unemp_features(df):
    d = df.sort_values(["county_fips","year"]).copy()
    if "unemployment_rate" not in d.columns:
        d["unemployment_rate"] = np.nan
    d["unemployment_rate"] = pd.to_numeric(d["unemployment_rate"], errors="coerce")
    g = d.groupby("county_fips")["unemployment_rate"]
    d["unemp_lag1"]  = g.shift(1)
    d["unemp_ma3"]   = g.transform(lambda s: s.rolling(3, min_periods=1).mean())
    d["unemp_chg_1y"]= d["unemployment_rate"] - d["unemp_lag1"]
    d["unemp_asof"]  = np.where(d["unemployment_rate"].notna(), d["unemployment_rate"], d["unemp_lag1"])
    d["unemp_missing_t"] = d["unemployment_rate"].isna().astype("Int8")
    return d

def add_poverty_features(df):
    d = df.sort_values(["county_fips","year"]).copy()
    if "poverty_rate" not in d.columns:
        d["poverty_rate"] = np.nan
    d["poverty_rate"] = pd.to_numeric(d["poverty_rate"], errors="coerce")
    g = d.groupby("county_fips")["poverty_rate"]
    d["pov_lag1"]   = g.shift(1)
    d["pov_ma3"]    = g.transform(lambda s: s.rolling(3, min_periods=1).mean())
    d["pov_chg_1y"] = d["poverty_rate"] - d["pov_lag1"]
    d["pov_asof"]   = np.where(d["poverty_rate"].notna(), d["poverty_rate"], d["pov_lag1"])
    d["pov_missing_t"] = d["poverty_rate"].isna().astype("Int8")
    return d

def add_crime_trends_if_present(df):
    d = df.sort_values(["county_fips","year"]).copy()
    for c in ["violent_crime_per_100k","property_crime_per_100k"]:
        if c in d.columns:
            g = d.groupby("county_fips")[c]
            d[c+"_lag1"]    = g.shift(1)
            d[c+"_pchg_3y"] = g.transform(lambda s: s.pct_change(3))
            d[c+"_pchg_5y"] = g.transform(lambda s: s.pct_change(5))
            d[c+"_missing_t"] = d[c].isna().astype("Int8")
    return d

def make_target(panel, hpi_col):
    d = panel.sort_values(["county_fips","year"]).copy()
    d["hpi_lead"] = d.groupby("county_fips")[hpi_col].shift(-1)
    d["y_growth_next"] = np.log(d["hpi_lead"]) - np.log(d[hpi_col])
    return d

# ---------- main entry ----------
def build_model_ready_dataset(panel_df: pd.DataFrame,
                              save=True,
                              winsorize_target=True) -> pd.DataFrame:
    """
    Create a clean, leak-safe dataset with features + next-year HPI target.
    No modeling here.
    """
    df = _ensure_keys(panel_df)
    hpi_col = _detect_hpi_col(df)

    # Features
    df = add_unemp_features(df)
    df = add_poverty_features(df)
    df = add_crime_trends_if_present(df)

    # Target
    df = make_target(df, hpi_col)

    # Select columns to keep
    feature_cols = [
        # labor
        "unemp_asof","unemp_lag1","unemp_chg_1y","unemp_ma3","unemp_missing_t",
        # poverty
        "pov_asof","pov_lag1","pov_chg_1y","pov_ma3","pov_missing_t",
    ]
    # optional crime/climate (keep if present)
    for c in ["violent_crime_per_100k","property_crime_per_100k"]:
        for s in ["", "_lag1", "_pchg_3y", "_pchg_5y", "_missing_t"]:
            col = c + s
            if col in df.columns: feature_cols.append(col)
    for c in ["fema_nri_overall_risk","nri_risk_flood","nri_risk_wildfire","heat_risk"]:
        if c in df.columns: feature_cols.append(c)

    keep = ["county_fips","year","y_growth_next", hpi_col] + feature_cols
    df = df[[c for c in keep if c in df.columns]].copy()

    # Basic QA / cleaning
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    # one row per key
    df = df.drop_duplicates(subset=["county_fips","year"])
    # winsorize target lightly (optional)
    if winsorize_target and "y_growth_next" in df.columns:
        df["y_growth_next_winz"] = _winsorize(df["y_growth_next"], 0.5, 99.5)

    # Coverage prints (optional)
    print("Unemployment as-of coverage (last 5 yrs):")
    print(_coverage_by_year(df, "unemp_asof").sort_index().tail(5).to_string())
    print("\nPoverty as-of coverage (last 5 yrs):")
    print(_coverage_by_year(df, "pov_asof").sort_index().tail(5).to_string())

    # Save
    if save:
        OUTDIR.mkdir(parents=True, exist_ok=True)
        stamp = datetime.datetime.now().strftime("%Y%m%d")
        out_parquet = OUTDIR / f"model_ready_{stamp}.parquet"
        out_csv     = OUTDIR / f"model_ready_{stamp}.csv"
        df.to_parquet(out_parquet, index=False)
        df.to_csv(out_csv, index=False)
        print("Saved:")
        print(" ", out_parquet)
        print(" ", out_csv)

    return df

# --- USAGE ---
# Assuming you already ran: merged_with_cc = add_crime_and_climate(merged_df, crime=..., climate=...)
# This only prepares the data. No modeling.
model_ready = build_model_ready_dataset(merged_with_cc, save=True)


Unemployment as-of coverage (last 5 yrs):
year
2020    100.0
2021    100.0
2022    100.0
2023    100.0
2024     99.7

Poverty as-of coverage (last 5 yrs):
year
2020    100.0
2021    100.0
2022    100.0
2023    100.0
2024     99.7
Saved:
  data\out\model_ready_20250822.parquet
  data\out\model_ready_20250822.csv


In [270]:
model_ready.head()

,county_fips,year,y_growth_next,zillow_index,unemp_asof,unemp_lag1,unemp_chg_1y,unemp_ma3,unemp_missing_t,pov_asof,pov_lag1,pov_chg_1y,pov_ma3,pov_missing_t,y_growth_next_winz
22622,01001,2001,0.021791,128468.753216,4.1,NaN,NaN,4.100000,0,10.8,NaN,NaN,10.800000,0,0.021791
22623,01001,2002,0.027732,131298.995264,4.8,4.1,0.7,4.450000,0,10.3,10.8,-0.5,10.550000,0,0.027732
22624,01001,2003,0.052465,134991.165015,5.0,4.8,0.2,4.633333,0,10.4,10.3,0.1,10.500000,0,0.052465
22625,01001,2004,0.073513,142262.582444,4.7,5.0,-0.3,4.833333,0,11.6,10.4,1.2,10.766667,0,0.073513
22626,01001,2005,0.065710,153114.666686,3.8,4.7,-0.9,4.500000,0,10.4,11.6,-1.2,10.800000,0,0.065710


In [272]:
import numpy as np
import pandas as pd
from pathlib import Path

# assumes you already have: model_ready (the table in your screenshot)
df = model_ready.sort_values(["county_fips","year"]).copy()

# --- add housing context from zillow_index ---
df["log_hpi_t"] = np.log(pd.to_numeric(df["zillow_index"], errors="coerce"))
df["hpi_growth_1y"] = df.groupby("county_fips")["log_hpi_t"].diff(1)

# --- choose target column (use winsorized if you prefer robustness) ---
y_col = "y_growth_next_winz" if "y_growth_next_winz" in df.columns else "y_growth_next"

# --- features to keep (only those that exist will be used) ---
feature_cols = [
    "unemp_asof","unemp_lag1","unemp_chg_1y","unemp_ma3","unemp_missing_t",
    "pov_asof","pov_lag1","pov_chg_1y","pov_ma3","pov_missing_t",
    "log_hpi_t","hpi_growth_1y",
]
feature_cols = [c for c in feature_cols if c in df.columns]

# --- final tidy table (IDs + target + features) ---
final = df[["county_fips","year", y_col] + feature_cols].copy()
final = final.replace([np.inf, -np.inf], np.nan)

# training rows for later (drop only missing TARGET; keep early-year lag NaNs)
train_ready = final.dropna(subset=[y_col]).reset_index(drop=True)

# save for another day
Path("data/out").mkdir(parents=True, exist_ok=True)
train_ready.to_parquet("data/out/model_ready_min.parquet", index=False)
train_ready.to_csv("data/out/model_ready_min.csv", index=False)

print("Saved:")
print(" data/out/model_ready_min.parquet")
print(" data/out/model_ready_min.csv")
print("Rows:", len(train_ready), " Features:", len(feature_cols))


Saved:
 data/out/model_ready_min.parquet
 data/out/model_ready_min.csv
Rows: 48233  Features: 12


In [276]:
print(df['year'].max())

2024
